In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
from scipy.stats import norm

from pylab import *
from pyomo.environ import *

### Question 1

Product Mix Problem

Your company grows two types of plants, A and B (e.g., roses and begonias).  

Both plant types require two types of fertilizer throughout the growing season, Fertilizer 1 and Fertilizer 2.
A single plant A will require 2 pounds of fertilizer 1 and 1 pound of fertilizer 2.
A single plant B will require 1 pound of fertilizer 1 and 2 pounds of fertilizer 2.
Your distributer has 4000 pounds of fertilizer 1 and 5000 pounds of fertilizer 2 available for delivery at the beginning of the season.

You know that every plant A you grow will ener dollarsate $2.25 profit, and every plant B you grow will ener dollars te $2.60 profit.

How much do you plant? (What is your business decision?) How much profit does this get you?


In [2]:
model = ConcreteModel("Roses and Begonias")

#DVs
model.r = Var(domain=NonNegativeIntegers)
model.b = Var(domain=NonNegativeIntegers)

#ObjFct
model.profit= Objective(expr = 2.25*model.r + 2.6*model.b, sense =maximize) 

#Cons
model.f1 = Constraint(expr = 2*model.r + model.b <= 4000)   
model.f2 = Constraint(expr = model.r + 2*model.b<= 5000)

SolverFactory('glpk').solve(model)  

model.display()

Model Roses and Begonias

  Variables:
    r : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1000.0 :  None : False : False : NonNegativeIntegers
    b : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2000.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7450.0

  Constraints:
    f1 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 4000.0 : 4000.0
    f2 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 5000.0 : 5000.0


In [3]:
print(f'''
Profit: {model.profit()}
R to plant: {model.r()}
B to plant: {model.b()}
F1: {model.f1()}
F2:{model.f2()}
''')


Profit: 7450.0
R to plant: 1000.0
B to plant: 2000.0
F1: 4000.0
F2:5000.0



### Question 2

Product Mix Continued

If your fertilizer storage capacity is 8000 pounds, what is your new product mix, and how much of each fertilizer should you order?  What would you be willing to pay for additional storage capacity?

Now how much do you plant?


In [5]:
model = ConcreteModel("Roses and Begonias Continued")

#DVs
model.r = Var(domain=NonNegativeIntegers) 
model.b = Var(domain=NonNegativeIntegers)

#ObjFct
model.profit= Objective(expr = 2.25*model.r + 2.6*model.b, sense =maximize) 

#Cons
model.f1 = Constraint(expr = 2*model.r + model.b <= 4000)  
model.f2 = Constraint(expr = model.r + 2*model.b<= 5000)
model.TotalStorage = Constraint(expr= 3*model.r+3*model.b <= 8000)

SolverFactory('glpk').solve(model)  

model.display()

Model Roses and Begonias Continued

  Variables:
    r : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 332.0 :  None : False : False : NonNegativeIntegers
    b : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2334.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 6815.400000000001

  Constraints:
    f1 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 2998.0 : 4000.0
    f2 : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 5000.0 : 5000.0
    TotalStorage : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 7998.0 : 8000.0


In [6]:
print(f'''
Profit: {model.profit()}
R to plant: {model.r()}
B to plant: {model.b()}
F1: {model.f1()}
F2:{model.f2()}
TotalStorage:{model.TotalStorage()}
''')


Profit: 6815.400000000001
R to plant: 332.0
B to plant: 2334.0
F1: 2998.0
F2:5000.0
TotalStorage:7998.0



### Question 3

The Dual Problem

Configure the Dual of the Product Mix Continued Problem, and find the solution.


In [2]:
model = ConcreteModel("Dual Problem Product Mix")

#DVs
model.f1Price = Var(domain=NonNegativeReals) 
model.f2Price = Var(domain=NonNegativeReals)
model.TotalStoragePrice = Var(domain=NonNegativeReals)

#ObjFct
model.constraint= Objective(expr = 4000*model.f1Price + 5000*model.f2Price + 8000*model.TotalStoragePrice, sense =minimize)

#Cons
model.R = Constraint(expr = 2*model.f1Price + model.f2Price + 3*model.TotalStoragePrice >= 2.25)  
model.B = Constraint(expr = 1*model.f1Price + 2*model.f2Price + 3*model.TotalStoragePrice >= 2.60)  

SolverFactory('glpk').solve(model)  #glpk is free to download

model.display()

Model Dual Problem Product Mix

  Variables:
    f1Price : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    f2Price : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  0.35 :  None : False : False : NonNegativeReals
    TotalStoragePrice : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 0.633333333333333 :  None : False : False : NonNegativeReals

  Objectives:
    constraint : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 6816.666666666663

  Constraints:
    R : Size=1
        Key  : Lower : Body              : Upper
        None :  2.25 : 2.249999999999999 :  None
    B : Size=1
        Key  : Lower : Body               : Upper
        None :   2.6 : 2.5999999999999988 :  None


In [13]:
print(f'''
Profit: {round(model.constraint(),2)}
F1 Shadow Price: {round(model.f1Price(),2)}
F2 Shadow Price: {round(model.f2Price(),2)}
Total Storage Shadow Price: {round(model.TotalStoragePrice(),2)}
R: {round(model.R(),3)}
B:{round(model.B(),3)}
''')


Profit: 6816.67
F1 Shadow Price: 0.0
F2 Shadow Price: 0.35
Total Storage Shadow Price: 0.63
R: 2.25
B:2.6



### Question 4

Product Mix Challenge

You manufacture two products, A and B, each of which you sell for $1 profit.  Product A requires 5 blobs and 3 globs, and product B requires 3 blobs and 5 globs.  Your supplier has 120 blobs and 120 globs available.  To maximize profit, how much of each product should you produce? How much profit can you make?


In [14]:
model = ConcreteModel("Product Mix Challenge")

#DVs
model.ProdA = Var(domain=NonNegativeIntegers) 
model.ProdB = Var(domain=NonNegativeIntegers)

#ObjFct
model.profit= Objective(expr = 1*model.ProdA + 1*model.ProdB, sense =maximize) 

#Cons
model.blobs = Constraint(expr = 5*model.ProdA + 3*model.ProdB <= 120)  
model.globs = Constraint(expr = 3*model.ProdA + 5*model.ProdB <= 120)

SolverFactory('glpk').solve(model)  

model.display()

Model Product Mix Challenge

  Variables:
    ProdA : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers
    ProdB : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  30.0

  Constraints:
    blobs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 120.0 : 120.0
    globs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 120.0 : 120.0


In [16]:
print(f'''
Profit: {model.profit()}
ProdA to produce: {model.ProdA()}
ProdB to produce: {model.ProdB()}
Blobs used: {model.blobs()}
Globs used:{model.globs()}
''')


Profit: 30.0
ProdA to produce: 15.0
ProdB to produce: 15.0
Blobs used: 120.0
Globs used:120.0



### Question 5

Golf Bags

Your company manufactures two types of golf bags, Standard and Deluxe.  You make $10 profit per Standard bag, and $9 profit per Deluxe bag that you sell.

Production times (in minutes) of each bag type are shown below, based on data you have collected from your assembly line.

    	Cutting and Dyeing	Sewing	Finishing	Inspection and Packaging
Standard	42	             30     	60	             6
Deluxe	    60	             50	        40	             15

Over the next 3 months, you estimate you have 630 hours of cutting and dyeing time, 600 hours of sewing time, 708 hours of finishing time, and 135 hours of inspection and packaging time available.

How many of each bag type should you manufacture over the next three months in order to maximize profit?


In [17]:
model = ConcreteModel("Golf Bags")

#DVs
model.Standard = Var(domain = NonNegativeIntegers)
model.Deluxe = Var(domain = NonNegativeIntegers)

#Objective
model.profit = Objective(expr= 10*model.Standard + 9*model.Deluxe, sense = maximize)

#Cons
model.CnD = Constraint(expr = 42*model.Standard + 60*model.Deluxe <= 37800)
model.S = Constraint(expr = 30*model.Standard + 50*model.Deluxe <= 36000)
model.F = Constraint(expr = 60*model.Standard + 40*model.Deluxe <= 42480)
model.InP = Constraint(expr = 6*model.Standard + 15*model.Deluxe <= 8100)

SolverFactory('glpk').solve(model)  

model.display()

Model Golf Bags

  Variables:
    Standard : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 540.0 :  None : False : False : NonNegativeIntegers
    Deluxe : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 252.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7668.0

  Constraints:
    CnD : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 37800.0 : 37800.0
    S : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 28800.0 : 36000.0
    F : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 42480.0 : 42480.0
    InP : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 7020.0 : 8100.0


In [19]:
print(f'''
Profit: {model.profit()}
Standard Golf Bags to produce: {model.Standard()}
Deluxe Golf Bags to produce: {model.Deluxe()}
Cutting and Dyeing minutes used: {model.CnD()}
Sewing minutes used:{model.S()}
Finishing minutes used:{model.F()}
Inspection and Packaging minutes used:{model.InP()}
''')


Profit: 7668.0
Standard Golf Bags to produce: 540.0
Deluxe Golf Bags to produce: 252.0
Cutting and Dyeing minutes used: 37800.0
Sewing minutes used:28800.0
Finishing minutes used:42480.0
Inspection and Packaging minutes used:7020.0



### Question 6

Baseball Gloves

Your company manufactures two types of baseball mitts, Fielder and Catcher.  The Fielder gloves generate $5 profit, and the Catcher gloves generate $8 profit.

Production times (in minutes) of each glove type are shown below, based on data you have collected from your assembly line.

	     Cutting and Sewing	     Finishing	  Packaging and Shipping
Fielder	     60	                    30	                12.5
Catcher	     90	                    20              	15

Over the next month, you estimate you have 900 hours of cutting and sewing time available, 300 hours of finishing time, and 100 hours of packaging and shipping time available.  

How many of each glove type should you manufacture over the next month in order to maximize profit?


In [23]:
model = ConcreteModel("Baseball Gloves")

#DVs

model.Fielder = Var(domain = NonNegativeIntegers)
model.Catcher = Var(domain = NonNegativeIntegers)

#Objective
model.profit = Objective(expr= 5*model.Fielder + 8*model.Catcher, sense = maximize)

#Cons
model.CnS = Constraint(expr = 60*model.Fielder + 90*model.Catcher <= 54000)
model.Fn = Constraint(expr = 30*model.Fielder + 20*model.Catcher <= 18000)
model.PnS = Constraint(expr = 12.5*model.Fielder + 15*model.Catcher <= 6000)

SolverFactory('glpk').solve(model)  

model.display()

Model Baseball Gloves

  Variables:
    Fielder : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    Catcher : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 400.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 3200.0

  Constraints:
    CnS : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 36000.0 : 54000.0
    Fn : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 8000.0 : 18000.0
    PnS : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 6000.0 : 6000.0


In [24]:
print(f'''
Profit: {model.profit()}
Fielder gloves to produce: {model.Fielder()}
Catcher gloves to produce: {model.Catcher()}
Cutting and Sewing minutes used: {model.CnS()}
Finishing minutes used:{model.Fn()}
Packaging and Shipping minutes used:{model.PnS()}
''')


Profit: 3200.0
Fielder gloves to produce: 0.0
Catcher gloves to produce: 400.0
Cutting and Sewing minutes used: 36000.0
Finishing minutes used:8000.0
Packaging and Shipping minutes used:6000.0



### Question 7

Bike Frames

Your company produces bike frames using two different material blend, Professional grade and Standard grade.  The cost per yard of the blends are $7.50 per yard for Standard and $9.00 per yard for Professional.  

The material content (in percent) of your blends are given in the following table:

	        Standard    Professional
Fiberglass	    0.84	   0.58
Carbon Fiber	0.1        0.3
Kevlar	        0.06	   0.12

You have a new contract to produce bike frames that require 30 total yards of material per frame.  The frames must have a carbon fiber content of at least 20%, and a Kevlar content of not more than 10%.  How many yards of each material blend (Standard and Professional) should go into each frame in order to minimize cost?


In [27]:
model = ConcreteModel("Bike Frames")

#DVs

model.Standard = Var(domain = NonNegativeIntegers)
model.Professional = Var(domain = NonNegativeIntegers)

#Objective
model.cost = Objective(expr= 7.5*model.Standard + 9*model.Professional, sense = minimize)

#Cons
model.TotalYards = Constraint(expr = model.Standard + model.Professional == 30)
model.CarbonFiber = Constraint(expr = 0.1*model.Standard + 0.3*model.Professional >= 30*0.2)
model.Kevlar = Constraint(expr = 0.06*model.Standard + 0.12*model.Professional <= 30*0.1)

SolverFactory('glpk').solve(model)  

model.display()

Model Bike Frames

  Variables:
    Standard : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers
    Professional : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 247.5

  Constraints:
    TotalYards : Size=1
        Key  : Lower : Body : Upper
        None :  30.0 : 30.0 :  30.0
    CarbonFiber : Size=1
        Key  : Lower : Body : Upper
        None :   6.0 :  6.0 :  None
    Kevlar : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 2.6999999999999997 :   3.0


In [33]:
print(f'''
Cost: {model.cost()}
Total yards of Standard Blend per Bike Frame: {model.Standard()}
Total yards of Professional Blend per Bike Frame: {model.Professional()}
Total yards: {model.TotalYards()}
Total CarbonFiber content:{model.CarbonFiber()}
Total Kevlar content:{round(model.Kevlar(),2)}
Total Fiberglass content: {model.TotalYards()-model.CarbonFiber()-model.Kevlar()}
''')


Cost: 247.5
Total yards of Standard Blend per Bike Frame: 15.0
Total yards of Professional Blend per Bike Frame: 15.0
Total yards: 30.0
Total CarbonFiber content:6.0
Total Kevlar content:2.7
Total Fiberglass content: 21.3



### Question 8

Investment Portfolios

Your company manages investments for your clients, where you build portfolios based on anticipated yield and risk.  (Yield and risk are generally based on historical performance of the investment instruments.)

The three instruments you are considering for a client are growth, income, and money market funds, which you determine to have risks of .1, .05, and .01 respectively.  Furthermore, you project the yields of these funds to be 20%, 10%, and 6% respectively.  

Your client insists that you diversify, with at least 10% in each of the growth and income funds, and at least 20% in the money market fund.

Your client wants to assume no more that 5% risk, where overall risk is calculated based on a weighted average of the risks of each individual instrument, and where the weightings are the percents of the investment that go towards each instrument.

If your client has $1,000,000 to invest, how should you allocate his funds?

Crossed out: How much can your yield estimates for each instrument change (individually) before you have to change this allocation?


In [37]:
model = ConcreteModel("Investment Portfolios")

#DVs

model.Gro = Var(domain = NonNegativeIntegers)
model.Inc = Var(domain = NonNegativeIntegers)
model.MMF = Var(domain = NonNegativeIntegers)


#Objective
model.yields = Objective(expr= 0.2*model.Gro + 0.1*model.Inc + .06*model.MMF, sense = maximize)

#Cons
model.TotalInv = Constraint(expr = model.Gro + model.Inc + model.MMF == 1000000)
model.Risk = Constraint(expr = 0.1*model.Gro + 0.05*model.Inc + 0.01*model.MMF <= 0.05*1000000)
model.MinGro = Constraint(expr = model.Gro >= 0.1*1000000)
model.MinInc = Constraint(expr = model.Inc >= 0.1*1000000)
model.MinMMF = Constraint(expr = model.MMF >= 0.2*1000000)
SolverFactory('glpk').solve(model)  

model.display()

Model Investment Portfolios

  Variables:
    Gro : Size=1, Index=None
        Key  : Lower : Value    : Upper : Fixed : Stale : Domain
        None :     0 : 400000.0 :  None : False : False : NonNegativeIntegers
    Inc : Size=1, Index=None
        Key  : Lower : Value    : Upper : Fixed : Stale : Domain
        None :     0 : 100000.0 :  None : False : False : NonNegativeIntegers
    MMF : Size=1, Index=None
        Key  : Lower : Value    : Upper : Fixed : Stale : Domain
        None :     0 : 500000.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    yields : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 120000.0

  Constraints:
    TotalInv : Size=1
        Key  : Lower     : Body      : Upper
        None : 1000000.0 : 1000000.0 : 1000000.0
    Risk : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 50000.0 : 50000.0
    MinGro : Size=1
        Key  : Lower    : Body     : Upper
        None : 100000.0 : 

In [38]:
print(f'''
Total yield: {model.yields()}
Investment in Growth Fund: {model.Gro()}
Investment in Income Fund: {model.Inc()}
Investment in Money Market Fund: {model.MMF()}
Total Risk:{model.Risk()}
''')


Total yield: 120000.0
Investment in Growth Fund: 400000.0
Investment in Income Fund: 100000.0
Investment in Money Market Fund: 500000.0
Total Risk:50000.0



### Question 9

Your company is prepared to undertake several new projects over the next five years.  Each Division has submitted a number of project proposals that include the annual budgets and anticipated profit.  

What project selection should your company choose?  (You decide what criterion to use, but recall that this is not the Daniels College of Philanthropy.)



In [48]:
profit = pd.read_excel('INFO4590P3PyomoData.xlsx',index_col=0)  

data = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="PSData",index_col=0)

In [49]:
profit

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15
Profit,1500,2000,2500,7000,4000,3000,4500,3500,1500,2000,2500,7000,4000,3000,4500


In [50]:
data

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,FUNDS
2016,400,500,200,300,450,650,350,300,500,600,150,250,220,170,420,3500
2017,230,270,430,220,500,450,530,700,800,250,500,340,400,300,400,4500
2018,330,260,430,270,400,320,330,260,430,270,400,320,250,300,260,4000
2019,400,500,200,300,450,650,350,300,500,600,150,250,220,170,420,3450
2020,230,270,430,220,500,450,530,700,800,250,500,340,270,300,400,4500


In [51]:
model = ConcreteModel()

P = profit.keys()  # keys=column headers, column can also be used instead of keys

model.x = Var(P,domain = Binary)

model.Profit = Objective(expr = sum([model.x[p]*profit.loc['Profit',p] for p in P]), sense = maximize) 


In [52]:
I = data.index  
model.cons = ConstraintList()

for i in I:
    model.cons.add(sum([model.x[p]*data.loc[i,p] for p in P]) <= data.loc[i,'FUNDS'])

In [53]:
SolverFactory('glpk').solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 43000.0, 'Upper bound': 43000.0, 'Number of objectives': 1, 'Number of constraints': 5, 'Number of variables': 15, 'Number of nonzeros': 75, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '25', 'Number of created subproblems': '25'}}, 'Error rc': 0, 'Time': 0.09816598892211914}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [54]:
model.display()

Model unknown

  Variables:
    x : Size=15, Index={P5, P4, P9, P12, P13, P6, P10, P2, P11, P15, P14, P1, P7, P3, P8}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         P1 :     0 :   0.0 :     1 : False : False : Binary
        P10 :     0 :   0.0 :     1 : False : False : Binary
        P11 :     0 :   0.0 :     1 : False : False : Binary
        P12 :     0 :   1.0 :     1 : False : False : Binary
        P13 :     0 :   1.0 :     1 : False : False : Binary
        P14 :     0 :   1.0 :     1 : False : False : Binary
        P15 :     0 :   1.0 :     1 : False : False : Binary
         P2 :     0 :   0.0 :     1 : False : False : Binary
         P3 :     0 :   1.0 :     1 : False : False : Binary
         P4 :     0 :   1.0 :     1 : False : False : Binary
         P5 :     0 :   1.0 :     1 : False : False : Binary
         P6 :     0 :   1.0 :     1 : False : False : Binary
         P7 :     0 :   1.0 :     1 : False : False : Binary
         P8 :     0 :   1.0 

### Question 10

Production Scheduling

Your company manufactures printer cases on two different injection molding machines.  The M-100 has a capacity of 25 cases per hour, and the M-200 has a capacity of 40 cases per hour.  

Both machines use the same chemicals in the manufacturing process; the M-100 uses 40 pounds of material per hour, and the M-200 uses 50 pounds per hour.

Your client has asked you to produce as many cases as possible next week, and they will pay $18 for every case you can deliver.

Unfortunately, you had scheduled maintenance down-time for both of your machines next week, so the M-100 will only be available for 15 hours maximum, and the M-200 will only be available for 10 hours maximum.  However, due to the high setup costs, if you run either machine during the week, they must run for a minimum of 5 hours.

The operating costs of the machines are $50/hour for the M-100 and $75/hour for the M-200.

Your chemical supplier has 1000 pounds of chemicals available next week at a cost of $6 per pound.  

What is your business decision for next week, and how much profit will it generate?


In [13]:
modelPrS = ConcreteModel("Production Scheduling")

#DVs
modelPrS.M100 = Var(domain=NonNegativeIntegers) 
modelPrS.M200 = Var(domain=NonNegativeIntegers)


In [14]:
#Obj Fct

modelPrS.profit= Objective(expr = (18-2-9.6)*modelPrS.M100 + (18-1.875-7.5)*modelPrS.M200, sense =maximize) 

In [15]:
#Cons
modelPrS.Chemical = Constraint(expr = 1.6*modelPrS.M100 + 1.25*modelPrS.M200 <= 1000)   
modelPrS.MinHours1 = Constraint(expr = modelPrS.M100>= 5*25)
modelPrS.MinHours2 = Constraint(expr = modelPrS.M200>= 5*40)
modelPrS.MaxHours1 = Constraint(expr = modelPrS.M100<= 15*25)
modelPrS.MaxHours2 = Constraint(expr = modelPrS.M200<= 10*40)


SolverFactory('glpk').solve(modelPrS)  


{'Problem': [{'Name': 'unknown', 'Lower bound': 5446.8, 'Upper bound': 5446.8, 'Number of objectives': 1, 'Number of constraints': 5, 'Number of variables': 2, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '3', 'Number of created subproblems': '3'}}, 'Error rc': 0, 'Time': 0.07184624671936035}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [16]:
modelPrS.display()

Model Production Scheduling

  Variables:
    M100 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 312.0 :  None : False : False : NonNegativeIntegers
    M200 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 400.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 5446.8

  Constraints:
    Chemical : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 999.2 : 1000.0
    MinHours1 : Size=1
        Key  : Lower : Body  : Upper
        None : 125.0 : 312.0 :  None
    MinHours2 : Size=1
        Key  : Lower : Body  : Upper
        None : 200.0 : 400.0 :  None
    MaxHours1 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 312.0 : 375.0
    MaxHours2 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 400.0 : 400.0


### Question 11

Assembly Line Equipment

In [2]:
cost = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="ALECost",index_col=0)  

data3 = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="ALEData",index_col=0)


In [3]:
cost

,M-I,M-II,M-III
Cost,18500,25000,35000


In [4]:
data3

,M-I,M-II,M-III,ProdGoal
Type1,100,265,200,3200
Type2,130,235,160,2500
Type3,140,170,260,3500
Type4,210,220,180,3000
Type5,80,120,220,2500


In [5]:
modelALE = ConcreteModel()

C = cost.keys()  

modelALE.x = Var(C,domain = NonNegativeIntegers)

modelALE.Cost = Objective(expr = sum([modelALE.x[c]*cost.loc['Cost',c] for c in C]), sense = minimize) 

In [6]:
I = data3.index  
modelALE.cons = ConstraintList()

for i in I:
    modelALE.cons.add(sum([modelALE.x[c]*data3.loc[i,c] for c in C]) >= data3.loc[i,'ProdGoal'])

In [7]:
SolverFactory('glpk').solve(modelALE)

{'Problem': [{'Name': 'unknown', 'Lower bound': 481000.0, 'Upper bound': 481000.0, 'Number of objectives': 1, 'Number of constraints': 5, 'Number of variables': 3, 'Number of nonzeros': 15, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '13', 'Number of created subproblems': '13'}}, 'Error rc': 0, 'Time': 0.07260608673095703}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
modelALE.display()

Model unknown

  Variables:
    x : Size=3, Index={M-II, M-III, M-I}
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
          M-I :     0 :   6.0 :  None : False : False : NonNegativeIntegers
         M-II :     0 :   5.0 :  None : False : False : NonNegativeIntegers
        M-III :     0 :   7.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    Cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 481000.0

  Constraints:
    cons : Size=5
        Key : Lower  : Body   : Upper
          1 : 3200.0 : 3325.0 :  None
          2 : 2500.0 : 3075.0 :  None
          3 : 3500.0 : 3510.0 :  None
          4 : 3000.0 : 3620.0 :  None
          5 : 2500.0 : 2620.0 :  None


### Question 12

Manufacturing and Distribution

In [3]:
ScostMD = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="MDSCost",index_col=0) 

McostMD = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="MDMCost",index_col=0) 

dataMD1 = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="MDData1",index_col=0)

dataMD2 = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="MDData2",index_col=0)

In [4]:
ScostMD

,Tacoma,San Diego,Dallas,Denver,St. Louis,Tampa,Baltimore
Macon,2.50,2.75,1.75,2.00,2.1,1.80,1.65
Louisville,1.85,1.90,1.50,1.60,1.0,1.90,1.85
Detroit,2.30,2.25,1.85,1.25,1.5,2.25,2.00
Phoenix,1.90,0.90,1.60,1.75,2.0,2.50,2.65


In [5]:
McostMD

,Cost/Unit
Macon,35.50
Louisville,37.50
Detroit,39.00
Phoenix,36.25


In [6]:
dataMD1

,Tacoma,San Diego,Dallas,Denver,St. Louis,Tampa,Baltimore
Total Demand,8500,14500,13500,12600,18000,15000,9000
Min Demand,6800,11600,10800,10080,14400,12000,7200


In [7]:
dataMD2

,Capacity
Macon,18000
Louisville,15000
Detroit,25000
Phoenix,20000


In [8]:
modelMD = ConcreteModel()

Ci = ScostMD.keys()  # distributor
P = ScostMD.index    # plant

#DVs
modelMD.x = Var( P,Ci,domain = NonNegativeIntegers)

#
modelMD.Cost = Objective(expr = sum(sum([modelMD.x[p,c] for c in Ci])*McostMD.loc[p,'Cost/Unit'] for p in P) + 
                         sum(sum([modelMD.x[p,c]*ScostMD.loc[p,c] for c in Ci]) for p in P), sense = minimize)
#modelCR.SCost = Objective(expr = sum(sum([modelMD.x[c,p]*costMD.loc['Cost',c] for c in Ci]) for p in P), sense = minimize)


C:\Users\amara\AppData\Local\Temp\ipykernel_9984\2163315576.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelMD.Cost = Objective(expr = sum(sum([modelMD.x[p,c] for c in Ci])*McostMD.loc[p,'Cost/Unit'] for p in P) + sum(sum([modelMD.x[p,c]*ScostMD.loc[p,c] for c in Ci]) for p in P), sense = minimize)
C:\Users\amara\AppData\Local\Temp\ipykernel_9984\2163315576.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelMD.Cost = Objective(expr = sum(sum([modelMD.x[p,c] for c in Ci])*McostMD.loc[p,'Cost/Unit'] for p in P) + sum(sum([modelMD.x[p,c]*ScostMD.loc[p,c] for c in Ci]) for p in P), sense = minimize)


In [9]:
#I1 = dataMD1.index 
modelMD.cons = ConstraintList()

for c in Ci:
    modelMD.cons.add(sum([modelMD.x[p,c] for p in P]) >= dataMD1.loc['Min Demand',c])

for p in P:
    modelMD.cons.add(sum([modelMD.x[p,c] for c in Ci])<= dataMD2.loc[p,'Capacity'])

In [10]:
SolverFactory('glpk').solve(modelMD)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2805450.0, 'Upper bound': 2805450.0, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 28, 'Number of nonzeros': 56, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.06254076957702637}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
modelMD.display()

Model unknown

  Variables:
    x : Size=28, Index={Phoenix, Macon, Detroit, Louisville}*{Denver, Tacoma, St. Louis, Dallas, Tampa, San Diego, Baltimore}
        Key                         : Lower : Value   : Upper : Fixed : Stale : Domain
           ('Detroit', 'Baltimore') :     0 :  1200.0 :  None : False : False : NonNegativeIntegers
              ('Detroit', 'Dallas') :     0 :  8600.0 :  None : False : False : NonNegativeIntegers
              ('Detroit', 'Denver') :     0 : 10080.0 :  None : False : False : NonNegativeIntegers
           ('Detroit', 'San Diego') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
           ('Detroit', 'St. Louis') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
              ('Detroit', 'Tacoma') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
               ('Detroit', 'Tampa') :     0 :     0.0 :  None : False : False : NonNegativeIntegers
        ('Louisville', 'Baltimore') :     0 :     0.0 :  No

### Question 13

Crushing More Rocks

In [47]:
costCR = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="CRCost",index_col=0)  

dataCR = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="CRData",index_col=0)

In [48]:
costCR

,Fine,Medium,Coarse
Cost,8,5,3


In [49]:
dataCR

,Fine,Medium,Coarse,Available
Limestone,0.5,0.2,0.05,50
Chat,0.3,0.4,0.20,60
Redi-Mix,0.2,0.3,0.35,70
Rough,0.0,0.1,0.40,30


In [50]:
modelCR = ConcreteModel()

C2 = costCR.keys()  

modelCR.x = Var(C2,domain = NonNegativeIntegers)

modelCR.Cost = Objective(expr = sum([modelCR.x[c]*costCR.loc['Cost',c] for c in C2]), sense = minimize) 

In [51]:
I = dataCR.index  
modelCR.cons = ConstraintList()

for i in I:
    modelCR.cons.add(sum([modelCR.x[c]*dataCR.loc[i,c] for c in C2]) >= dataCR.loc[i,'Available'])
for c in C2:
    modelCR.cons.add(modelCR.x[c] >=50)

In [52]:
SolverFactory('glpk').solve(modelCR)

{'Problem': [{'Name': 'unknown', 'Lower bound': 1153.0, 'Upper bound': 1153.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 3, 'Number of nonzeros': 14, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '5', 'Number of created subproblems': '5'}}, 'Error rc': 0, 'Time': 0.06146883964538574}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [53]:
modelCR.display()

Model unknown

  Variables:
    x : Size=3, Index={Coarse, Fine, Medium}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        Coarse :     0 : 118.0 :  None : False : False : NonNegativeIntegers
          Fine :     0 :  68.0 :  None : False : False : NonNegativeIntegers
        Medium :     0 :  51.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    Cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1153.0

  Constraints:
    cons : Size=7
        Key : Lower : Body               : Upper
          1 :  50.0 :               50.1 :  None
          2 :  60.0 :               64.4 :  None
          3 :  70.0 :  70.19999999999999 :  None
          4 :  30.0 : 52.300000000000004 :  None
          5 :  50.0 :               68.0 :  None
          6 :  50.0 :               51.0 :  None
          7 :  50.0 :              118.0 :  None


### Question 14

Hospital Scheduling

In [54]:
profitHS = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="HSProfit",index_col=0)  

dataHS = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="HSData",index_col=0)


In [55]:
profitHS

,Face Lift,Lipo,Implants
Profit,240,225,425


In [56]:
dataHS

,Face Lift,Lipo,Implants,Available
Days of Stay,3,5.0,6,490
Surgical Suite Hours,2,1.5,3,165


In [58]:
modelHS = ConcreteModel()

PHS = profitHS.keys()  

modelHS.x = Var(PHS,domain = NonNegativeIntegers)

modelHS.Profit = Objective(expr = sum([modelHS.x[p]*profitHS.loc['Profit',p] for p in PHS]), sense = maximize)

In [59]:
IHS = dataHS.index  
modelHS.cons = ConstraintList()

for i in IHS:
    modelHS.cons.add(sum([modelHS.x[p]*dataHS.loc[i,p] for p in PHS]) <= dataHS.loc[i,'Available'])


In [60]:
SolverFactory('glpk').solve(modelHS)

{'Problem': [{'Name': 'unknown', 'Lower bound': 24375.0, 'Upper bound': 24375.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 3, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.07189297676086426}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [61]:
modelHS.display()

Model unknown

  Variables:
    x : Size=3, Index={Lipo, Face Lift, Implants}
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
        Face Lift :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         Implants :     0 :  15.0 :  None : False : False : NonNegativeIntegers
             Lipo :     0 :  80.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    Profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 24375.0

  Constraints:
    cons : Size=2
        Key : Lower : Body  : Upper
          1 :  None : 490.0 : 490.0
          2 :  None : 165.0 : 165.0


### Question 15

Butter

In [17]:
profitB = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="BProfit",index_col=0)  

dataB = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="BData",index_col=0)


In [18]:
profitB

,PB,AB
Profit,1.1,1.3


In [19]:
dataB

,PB,AB,Available
Packaging Capacity,0.005,0.004,40
Sterilization Capacity,0.004,0.006,40


In [20]:
modelB = ConcreteModel()
BP = profitB.keys()

modelB.x = Var(BP,domain = NonNegativeIntegers)

modelB.Profit = Objective(expr = sum([modelB.x[p]*profitB.loc['Profit',p] for p in BP]), sense = maximize)


In [21]:
IB = dataB.index  
modelB.cons = ConstraintList()

for i in IB:
    modelB.cons.add(sum([modelB.x[p]*dataB.loc[i,p] for p in BP]) <= dataB.loc[i,'Available'])
modelB.cons.add(modelB.x['AB']>=5000)

In [22]:
SolverFactory('glpk').solve(modelB)

{'Problem': [{'Name': 'unknown', 'Lower bound': 9250.0, 'Upper bound': 9250.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of nonzeros': 5, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.11942028999328613}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [23]:
modelB.display()

Model unknown

  Variables:
    x : Size=2, Index={PB, AB}
        Key : Lower : Value  : Upper : Fixed : Stale : Domain
         AB :     0 : 5000.0 :  None : False : False : NonNegativeIntegers
         PB :     0 : 2500.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    Profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 9250.0

  Constraints:
    cons : Size=3
        Key : Lower  : Body   : Upper
          1 :   None :   32.5 :  40.0
          2 :   None :   40.0 :  40.0
          3 : 5000.0 : 5000.0 :  None


### Question 16

Pet Food

In [28]:
RevenuePF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="PFRevenue",index_col=0)  
CostPF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="PFCost",index_col=0)  
CompositionPF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="PFComposition",index_col=0)  
DepartmentPF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="PFDepartment",index_col=0) 


In [29]:
RevenuePF

,BFSeed,BFStones,BFCereal,DFMeat,DFFishmeal,DFCereal
Revenue,750,750,750,980,980,980


In [30]:
CostPF

,BFSeed,BFStones,BFCereal,DFMeat,DFFishmeal,DFCereal
Cost,700,100,200,600,900,200


In [31]:
CompositionPF

,BFSeed,BFStones,BFCereal,DFMeat,DFFishmeal,DFCereal,BFMin,DFMin
Protein,0.10,0.00,0.03,0.12,0.20,0.03,0.05,0.11
Carbohydrates,0.10,0.00,0.30,0.10,0.08,0.30,0.18,0.15
Trace Minerals,0.02,0.03,0.00,0.01,0.02,0.00,0.01,0.01
Abrasives,0.01,1.00,0.00,0.00,0.02,0.00,0.02,0.00


In [32]:
DepartmentPF

,BFSeed,BFStones,BFCereal,DFMeat,DFFishmeal,DFCereal,Available
Blending,0.25,0.25,0.25,0.15,0.15,0.15,8
Packaging,0.10,0.10,0.10,0.30,0.30,0.30,8


In [34]:
modelPF = ConcreteModel()
DV = RevenuePF.keys()    

#DVs

modelPF.x = Var(DV,domain = NonNegativeReals)

#Objective Function

modelPF.Profit = Objective(expr = sum([modelPF.x[v]*RevenuePF.loc['Revenue',v] for v in DV]) -
                           sum([modelPF.x[v]*CostPF.loc['Cost',v] for v in DV]), sense = maximize) 



In [35]:
Mat = CompositionPF.index 
Dep = DepartmentPF.index
modelPF.cons = ConstraintList()

for d in Dep:
    modelPF.cons.add(sum([modelPF.x[v]*DepartmentPF.loc[d,v] for v in DV]) <= DepartmentPF.loc[d,'Available'])

TotalBF= modelPF.x['BFSeed'] + modelPF.x['BFStones'] + modelPF.x['BFCereal']
TotalDF= modelPF.x['DFMeat'] + modelPF.x['DFFishmeal'] + modelPF.x['DFCereal']


for m in Mat:
    modelPF.cons.add( (modelPF.x['BFSeed']*CompositionPF.loc[m,'BFSeed'] 
                       + modelPF.x['BFStones']*CompositionPF.loc[m,'BFStones']
                       + modelPF.x['BFCereal']*CompositionPF.loc[m,'BFCereal'] )  >= CompositionPF.loc[m,'BFMin']* TotalBF)

for m in Mat:
    modelPF.cons.add( (modelPF.x['DFMeat']*CompositionPF.loc[m,'DFMeat'] 
                       + modelPF.x['DFFishmeal']*CompositionPF.loc[m,'DFFishmeal']
                       + modelPF.x['DFCereal']*CompositionPF.loc[m,'DFCereal'] )  >= CompositionPF.loc[m,'DFMin']*TotalDF)
    
modelPF.cons.add(modelPF.x['BFSeed'] >= .1*TotalBF)

In [36]:
SolverFactory('glpk').solve(modelPF)

{'Problem': [{'Name': 'unknown', 'Lower bound': 16488.8888888889, 'Upper bound': 16488.8888888889, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 6, 'Number of nonzeros': 36, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.07813811302185059}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [37]:
modelPF.display()

Model unknown

  Variables:
    x : Size=6, Index={BFCereal, BFSeed, DFCereal, DFFishmeal, DFMeat, BFStones}
        Key        : Lower : Value            : Upper : Fixed : Stale : Domain
          BFCereal :     0 : 11.1111111111111 :  None : False : False : NonNegativeReals
            BFSeed :     0 : 6.66666666666667 :  None : False : False : NonNegativeReals
          BFStones :     0 : 2.22222222222222 :  None : False : False : NonNegativeReals
          DFCereal :     0 :             10.0 :  None : False : False : NonNegativeReals
        DFFishmeal :     0 :             10.0 :  None : False : False : NonNegativeReals
            DFMeat :     0 :              0.0 :  None : False : False : NonNegativeReals

  Objectives:
    Profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 16488.888888888883

  Constraints:
    cons : Size=11
        Key : Lower : Body                   : Upper
          1 :  None :      7.999999999999997 :   8.0
    

### Question 17

Employee Scheduling

In [3]:
CostES1 = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="ES1Cost",index_col=0)  
DataES1 = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="ES1Data",index_col=0)  


In [4]:
CostES1

,S-1,S-2,S-3,S-4,S-5,S-6,S-7
Cost,680,705,705,705,705,680,655


In [5]:
DataES1

,S-1,S-2,S-3,S-4,S-5,S-6,S-7,Demand (packages)
Tues,1,0,0,1,1,1,1,21101.467706
Wed,1,1,0,0,1,1,1,24919.494949
Thurs,1,1,1,0,0,1,1,23554.342348
Fri,1,1,1,1,0,0,1,20388.683176
Sat,1,1,1,1,1,0,0,17707.072963
Sun,0,1,1,1,1,1,0,17302.201179
Mon,0,0,1,1,1,1,1,25631.132594


In [6]:
modelES1 = ConcreteModel ("Employee Scheduling-1")

Shift = CostES1.keys()

#DVs

modelES1.x = Var(Shift,domain = NonNegativeIntegers)

#Objective Function
modelES1.Cost = Objective(expr = sum([modelES1.x[s]*CostES1.loc['Cost',s] for s in Shift]), sense = minimize)

In [13]:
Days = DataES1.index 

modelES1.cons = ConstraintList()

for d in Days:
    modelES1.cons.add(1000*sum([modelES1.x[s]*DataES1.loc[d,s] for s in Shift]) >= DataES1.loc[d,'Demand (packages)'])

'pyomo.core.base.constraint.ConstraintList'>) on block Employee Scheduling-1
with a new Component (type=<class
'pyomo.core.base.constraint.ConstraintList'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [14]:
SolverFactory('glpk').solve(modelES1)

{'Problem': [{'Name': 'unknown', 'Lower bound': 21205.0, 'Upper bound': 21205.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 7, 'Number of nonzeros': 35, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '13', 'Number of created subproblems': '13'}}, 'Error rc': 0, 'Time': 0.10185074806213379}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [15]:
modelES1.display()

Model Employee Scheduling-1

  Variables:
    x : Size=7, Index={S-2, S-3, S-4, S-7, S-5, S-6, S-1}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        S-1 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
        S-2 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
        S-3 :     0 :   6.0 :  None : False : False : NonNegativeIntegers
        S-4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        S-5 :     0 :   7.0 :  None : False : False : NonNegativeIntegers
        S-6 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
        S-7 :     0 :  10.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    Cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 21205.0

  Constraints:
    cons : Size=7
        Key : Lower              : Body    : Upper
          1 : 21101.467706169988 : 23000.0 :  None
          2 : 24919.494948545263 : 25000.0 :  None
          3 : 

### Question 18

Lockbox Problem

In [26]:
FloatLB = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="LBFloatDays",index_col=0)  
PaymentLB = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="LBPayment",index_col=0)  
CostLB = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="LBCost",index_col=0)  


In [27]:
FloatLB

,Sacramento,Denver,Chicago,Dallas,New York,Atlanta
Central,4,2,2,2,3,3
Mid-Atlantic,6,4,3,4,2,2
Midwest,3,2,3,2,5,4
Northeast,6,4,2,5,2,3
Northwest,2,3,5,4,6,7
Southeast,7,4,3,2,4,2
Southwest,2,3,6,2,7,6


In [28]:
PaymentLB

,Sacramento,Denver,Chicago,Dallas,New York,Atlanta
Central,45,45,45,45,45,45
Mid-Atlantic,65,65,65,65,65,65
Midwest,50,50,50,50,50,50
Northeast,90,90,90,90,90,90
Northwest,70,70,70,70,70,70
Southeast,80,80,80,80,80,80
Southwest,60,60,60,60,60,60


In [29]:
CostLB

,Sacramento,Denver,Chicago,Dallas,New York,Atlanta
Cost,25,60,35,35,30,35


In [45]:
modelLB = ConcreteModel ("Lockbox Problem")

Location = FloatLB.keys()
Region = FloatLB.index


#DVs

modelLB.x = Var(Region,Location,domain = Binary)
modelLB.y = Var(Location,domain = Binary)

#Objective Function

modelLB.Cost = Objective(expr = 1000*0.15*sum(sum([modelLB.x[r,l]*FloatLB.loc[r,l]*PaymentLB.loc[r,l] for l in Location]) for r in Region) +
                         1000*sum([modelLB.y[l]*CostLB.loc['Cost',l] for l in Location]), sense = minimize)

C:\Users\amara\AppData\Local\Temp\ipykernel_17092\497191480.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelLB.Cost = Objective(expr = 1000*0.15*sum(sum([modelLB.x[r,l]*FloatLB.loc[r,l]*PaymentLB.loc[r,l] for l in Location]) for r in Region) +


In [46]:
modelLB.cons = ConstraintList()

for r in Region:
    modelLB.cons.add(sum([modelLB.x[r,l] for l in Location]) ==1)

for r in Region:
    for l in Location:
        modelLB.cons.add(modelLB.x[r,l] <= modelLB.y[l])

In [47]:
SolverFactory('glpk').solve(modelLB)

{'Problem': [{'Name': 'unknown', 'Lower bound': 224000.0, 'Upper bound': 224000.0, 'Number of objectives': 1, 'Number of constraints': 49, 'Number of variables': 48, 'Number of nonzeros': 126, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.10447311401367188}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [48]:
modelLB.display()

Model Lockbox Problem

  Variables:
    x : Size=42, Index={Southeast, Northwest, Southwest, Central, Northeast, Mid-Atlantic, Midwest}*{Atlanta, Dallas, Sacramento, Chicago, Denver, New York }
        Key                            : Lower : Value : Upper : Fixed : Stale : Domain
                ('Central', 'Atlanta') :     0 :   0.0 :     1 : False : False : Binary
                ('Central', 'Chicago') :     0 :   0.0 :     1 : False : False : Binary
                 ('Central', 'Dallas') :     0 :   1.0 :     1 : False : False : Binary
                 ('Central', 'Denver') :     0 :   0.0 :     1 : False : False : Binary
              ('Central', 'New York ') :     0 :   0.0 :     1 : False : False : Binary
             ('Central', 'Sacramento') :     0 :   0.0 :     1 : False : False : Binary
           ('Mid-Atlantic', 'Atlanta') :     0 :   0.0 :     1 : False : False : Binary
           ('Mid-Atlantic', 'Chicago') :     0 :   0.0 :     1 : False : False : Binary
            ('

### Question 19

Farming

In [31]:
ProfitF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="FProfit",index_col=0)  
Data1F = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="FData1",index_col=0)  
Data2F = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="FData2",index_col=0)  


In [32]:
ProfitF

,Milo,Cotton,Wheat
F-1,400,300,100
F-2,400,300,100
F-3,400,300,100


In [33]:
Data1F

,Acreage,Water Available (acre feet)
F-1,400,1500
F-2,600,2000
F-3,300,900


In [34]:
Data2F

,Milo,Cotton,Wheat
Total Harvesting Capacity (in acres),700,800,300
Water Requirements (in acres-feet/acre),6,4,2


In [35]:
modelF = ConcreteModel ("Farming")

Crop = ProfitF.keys()
Farm = ProfitF.index

#DVs

modelF.x = Var(Farm,Crop,domain = NonNegativeIntegers)

#Objective Function

modelF.Profit = Objective(expr = sum(sum([modelF.x[f,c]*ProfitF.loc[f,c] for c in Crop]) for f in Farm), sense = maximize)

C:\Users\amara\AppData\Local\Temp\ipykernel_35388\3758790818.py:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelF.Profit = Objective(expr = sum(sum([modelF.x[f,c]*ProfitF.loc[f,c] for c in Crop]) for f in Farm), sense = maximize)


In [37]:
modelF.cons = ConstraintList()

for f in Farm:
    modelF.cons.add(sum([modelF.x[f,c] for c in Crop]) <= Data1F.loc[f,'Acreage'])

for c in Crop:
     modelF.cons.add(sum([modelF.x[f,c] for f in Farm]) <= Data2F.loc['Total Harvesting Capacity (in acres)',c])

for f in Farm:
    modelF.cons.add(sum([modelF.x[f,c]*Data2F.loc['Water Requirements (in acres-feet/acre)',c] for c in Crop]) <= Data1F.loc[f,'Water Available (acre feet)'])

'pyomo.core.base.constraint.ConstraintList'>) on block Farming with a new
Component (type=<class 'pyomo.core.base.constraint.ConstraintList'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [38]:
SolverFactory('glpk').solve(modelF)

{'Problem': [{'Name': 'unknown', 'Lower bound': 320000.0, 'Upper bound': 320000.0, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 9, 'Number of nonzeros': 27, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.08519792556762695}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [39]:
modelF.display()

Model Farming

  Variables:
    x : Size=9, Index={F-2, F-3, F-1}*{Wheat , Cotton , Milo }
        Key                : Lower : Value : Upper : Fixed : Stale : Domain
        ('F-1', 'Cotton ') :     0 : 375.0 :  None : False : False : NonNegativeIntegers
          ('F-1', 'Milo ') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         ('F-1', 'Wheat ') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('F-2', 'Cotton ') :     0 : 200.0 :  None : False : False : NonNegativeIntegers
          ('F-2', 'Milo ') :     0 : 200.0 :  None : False : False : NonNegativeIntegers
         ('F-2', 'Wheat ') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('F-3', 'Cotton ') :     0 : 225.0 :  None : False : False : NonNegativeIntegers
          ('F-3', 'Milo ') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         ('F-3', 'Wheat ') :     0 :   0.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    Profit : Size

### Question 20

Inventory Management

In [14]:
DataIM = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="IMData",index_col=0)  


In [15]:
DataIM

,DemL,DemH
D-1,6,0
D-2,3,2
D-3,5,4
D-4,8,3
D-5,2,1
D-6,4,4
D-7,1,2


In [18]:
modelIM = ConcreteModel ("Inventory Management- Trucks")

Days = IMDV.index

#DVs

modelIM.Light = Var(domain = NonNegativeIntegers)
modelIM.Heavy = Var(domain = NonNegativeIntegers)

modelIM.LightsUsed = Var(Days, domain = NonNegativeIntegers)
modelIM.LightsRented = Var(Days, domain = NonNegativeIntegers)
modelIM.HeavyForLight = Var(Days, domain = NonNegativeIntegers)
modelIM.HeavyUsed = Var(Days, domain = NonNegativeIntegers)
modelIM.HeavyRented = Var(Days, domain = NonNegativeIntegers)


#Objective Function

modelIM.Cost = Objective(expr = sum([32*modelIM.Light + 44*modelIM.Heavy + 
                                     40*modelIM.LightsUsed[d] + 
                                     54*(modelIM.HeavyUsed[d] + modelIM.HeavyForLight[d]) + 
                                     175*modelIM.LightsRented[d] + 225*modelIM.HeavyRented[d] for d in Days]) ,sense = minimize)

In [19]:
modelIM.cons = ConstraintList()

for d in Days:
    modelIM.cons.add(modelIM.HeavyUsed[d] + modelIM.HeavyRented[d] >= DataIM.loc[d,'DemH'])
for d in Days:
    modelIM.cons.add(modelIM.HeavyUsed[d] + modelIM.HeavyForLight[d] <= modelIM.Heavy)
    
for d in Days:
    modelIM.cons.add(modelIM.LightsUsed[d] + modelIM.HeavyForLight[d] + modelIM.LightsRented[d] >= DataIM.loc[d,'DemL'])
for d in Days:
    modelIM.cons.add(modelIM.LightsUsed[d] <= modelIM.Light)


In [20]:
SolverFactory('glpk').solve(modelIM)

{'Problem': [{'Name': 'unknown', 'Lower bound': 4674.0, 'Upper bound': 4674.0, 'Number of objectives': 1, 'Number of constraints': 28, 'Number of variables': 37, 'Number of nonzeros': 70, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.07364201545715332}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [21]:
modelIM.display()

Model Inventory Management- Trucks

  Variables:
    Light : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeIntegers
    Heavy : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   4.0 :  None : False : False : NonNegativeIntegers
    LightsUsed : Size=7, Index={D-4, D-1, D-3, D-5, D-7, D-6, D-2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        D-1 :     0 :   5.0 :  None : False : False : NonNegativeIntegers
        D-2 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
        D-3 :     0 :   5.0 :  None : False : False : NonNegativeIntegers
        D-4 :     0 :   5.0 :  None : False : False : NonNegativeIntegers
        D-5 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
        D-6 :     0 :   4.0 :  None : False : False : NonNegativeIntegers
        D-7 :     0 :   1.0 :  None : False

### Question 21

Inventory Depletion

In [43]:
ProfitID = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="IDProfit",index_col=0)  
DataID = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="IDData",index_col=0)  


In [44]:
ProfitID

,Weekender,Expedition
SPrice,3.8,7.0
HSCost,1.5,1.5


In [45]:
DataID

,Weekender,Expedition,Total Inventory (in ounces)
Dried Fruit,3,5,10000
Dried Meat,7,18,25000
Dried Vegetables,2,5,12000


In [46]:
modelID = ConcreteModel ("Inventory Depletion")

Product = ProfitID.keys()

#DVs

modelID.x = Var(Product,domain = NonNegativeIntegers)

#Objective Function

modelID.Profit = Objective(expr = sum([modelID.x[p]*ProfitID.loc['SPrice',p] for p in Product])-
                           sum([modelID.x[p]*ProfitID.loc['HSCost',p] for p in Product]), sense = maximize)

In [47]:
Ingredient = DataID.index

modelID.cons = ConstraintList()

for i in Ingredient:
    modelID.cons.add(sum([modelID.x[p]*DataID.loc[i,p] for p in Product]) <= DataID.loc[i,'Total Inventory (in ounces)'])

In [48]:
SolverFactory('glpk').solve(modelID)

{'Problem': [{'Name': 'unknown', 'Lower bound': 8105.0, 'Upper bound': 8105.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.06512761116027832}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [49]:
modelID.display()

Model Inventory Depletion

  Variables:
    x : Size=2, Index={Expedition, Weekender}
        Key        : Lower : Value  : Upper : Fixed : Stale : Domain
        Expedition :     0 :  263.0 :  None : False : False : NonNegativeIntegers
         Weekender :     0 : 2895.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    Profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 8105.0

  Constraints:
    cons : Size=3
        Key : Lower : Body    : Upper
          1 :  None : 10000.0 : 10000.0
          2 :  None : 24999.0 : 25000.0
          3 :  None :  7105.0 : 12000.0


### Question 22

Son of Employee Scheduling

In [88]:
CostES = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="ESCost",index_col=0)  
DataES = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="ESData",index_col=0)  


In [89]:
CostES

,S-7,S-8,S-9,S-10,S-11
EngConsultant,1.0,1.0,1.0,1.0,1.0
BilConsultant,1.1,1.1,1.1,1.1,1.1


In [90]:
DataES

,S-7,S-8,S-9,S-10,S-11,Required Spanish,Required Total Consultants
H-7,1,0,0,0,0,5,9
H-8,1,1,0,0,0,5,9
H-9,1,1,1,0,0,4,9
H-10,1,1,1,1,0,3,9
H-11,0,1,1,1,1,2,8
H-12,1,0,1,1,1,3,11
H-13,1,1,0,1,1,4,9
H-14,1,1,1,0,1,3,7
H-15,1,1,1,1,0,2,6
H-16,0,1,1,1,1,1,6


In [93]:
modelES = ConcreteModel ("Employee Scheduling")

S = CostES.keys()
Con = CostES.index

#DVs

modelES.x = Var(Con,S,domain = NonNegativeIntegers)

#Objective Function
modelES.Cost = Objective(expr = sum(sum([modelES.x[c,s]*CostES.loc[c,s] for c in Con]) for s in S), sense = minimize)

C:\Users\amara\AppData\Local\Temp\ipykernel_9984\2998292125.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelES.Cost = Objective(expr = sum(sum([modelES.x[c,s]*CostES.loc[c,s] for c in Con]) for s in S), sense = minimize)


In [94]:
Hours = DataES.index 

modelES.cons = ConstraintList()

for h in Hours:
    modelES.cons.add(sum([modelES.x['BilConsultant',s]*DataES.loc[h,s] for s in S])>= DataES.loc[h,'Required Spanish'])

for h in Hours:
    modelES.cons.add(sum([sum(modelES.x[c,s] for c in Con)*DataES.loc[h,s] for s in S]) >= DataES.loc[h,'Required Total Consultants'])

C:\Users\amara\AppData\Local\Temp\ipykernel_9984\2906129507.py:9: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelES.cons.add(sum([sum(modelES.x[c,s] for c in Con)*DataES.loc[h,s] for s in S]) >= DataES.loc[h,'Required Total Consultants'])


In [95]:
SolverFactory('glpk').solve(modelES)

{'Problem': [{'Name': 'unknown', 'Lower bound': 18.9, 'Upper bound': 18.9, 'Number of objectives': 1, 'Number of constraints': 26, 'Number of variables': 10, 'Number of nonzeros': 120, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.08024191856384277}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [96]:
modelES.display()

Model Employee Scheduling

  Variables:
    x : Size=10, Index={EngConsultant, BilConsultant}*{S-7, S-11, S-8, S-9, S-10}
        Key                       : Lower : Value : Upper : Fixed : Stale : Domain
        ('BilConsultant', 'S-10') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('BilConsultant', 'S-11') :     0 :   4.0 :  None : False : False : NonNegativeIntegers
         ('BilConsultant', 'S-7') :     0 :   5.0 :  None : False : False : NonNegativeIntegers
         ('BilConsultant', 'S-8') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         ('BilConsultant', 'S-9') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('EngConsultant', 'S-10') :     0 :   2.0 :  None : False : False : NonNegativeIntegers
        ('EngConsultant', 'S-11') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
         ('EngConsultant', 'S-7') :     0 :   4.0 :  None : False : False : NonNegativeIntegers
         ('EngConsultant', 

### Question 23

Multiperiod Planning

In [107]:
ProfitMPP = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="MPPProfit",index_col=0)  


In [108]:
ProfitMPP

,Revenue,Cost,Storage cost
Chalet PM1,80,20,0
Chalet WM1,80,35,0
Chalet AM1,80,50,0
Hovel PM1,40,20,0
Hovel WM1,40,35,0
Hovel AM1,40,50,0
Chalet PM12,81,20,2
Chalet WM12,81,35,2
Chalet AM12,81,50,2
Hovel PM12,39,20,2


In [118]:
modelMPP = ConcreteModel ("Multiperiod Planning")

ProdDV = ProfitMPP.index

#DVs

modelMPP.x = Var(ProdDV,domain = NonNegativeReals)

#Objective Function
modelMPP.Profit = Objective(expr = sum([modelMPP.x[p]*ProfitMPP.loc[p,'Revenue'] for p in ProdDV])-
                            sum([modelMPP.x[p]*ProfitMPP.loc[p,'Cost'] for p in ProdDV])-
                            sum([modelMPP.x[p]*ProfitMPP.loc[p,'Storage cost'] for p in ProdDV]), sense = maximize)

In [119]:

TotalChaletM1 = modelMPP.x['Chalet PM1'] + modelMPP.x['Chalet WM1'] + modelMPP.x['Chalet AM1']
TotalChaletM2 = modelMPP.x['Chalet PM12'] + modelMPP.x['Chalet WM12'] + modelMPP.x['Chalet AM12'] + modelMPP.x['Chalet PM2'] + modelMPP.x['Chalet WM2'] + modelMPP.x['Chalet AM2']

TotalHovelM1 = modelMPP.x['Hovel PM1'] + modelMPP.x['Hovel WM1'] + modelMPP.x['Hovel AM1']
TotalHovelM2 = modelMPP.x['Hovel PM12'] + modelMPP.x['Hovel WM12'] + modelMPP.x['Hovel AM12'] + modelMPP.x['Hovel PM2'] + modelMPP.x['Hovel WM2'] + modelMPP.x['Hovel AM2']


In [120]:
modelMPP.cons = ConstraintList()

#Product Mix Constraints
modelMPP.cons.add(TotalChaletM1 + TotalHovelM1 <= 700)
modelMPP.cons.add(TotalChaletM2 + TotalHovelM2 <= 700)

#Demand Constraint
modelMPP.cons.add(TotalChaletM2 >= 200)

#Nuts Availability Constraints
modelMPP.cons.add(modelMPP.x['Chalet PM2'] +  modelMPP.x['Hovel PM2'] <= 500)
modelMPP.cons.add(modelMPP.x['Chalet AM2'] +  modelMPP.x['Hovel AM2'] <= 180)
modelMPP.cons.add(modelMPP.x['Chalet PM1'] +  modelMPP.x['Hovel PM1'] + modelMPP.x['Chalet PM12'] +  modelMPP.x['Hovel PM12'] <= 400)
modelMPP.cons.add(modelMPP.x['Chalet AM1'] +  modelMPP.x['Hovel AM1'] + modelMPP.x['Chalet AM12'] +  modelMPP.x['Hovel AM12']<= 200)

#Peanuts Composition Constraints
modelMPP.cons.add(modelMPP.x['Chalet PM1'] <= TotalChaletM1*0.25)
modelMPP.cons.add(modelMPP.x['Chalet PM12'] + modelMPP.x['Chalet PM2'] <= TotalChaletM2*0.25)

modelMPP.cons.add(modelMPP.x['Hovel PM1'] <= TotalHovelM1*0.6)
modelMPP.cons.add(modelMPP.x['Hovel PM12'] + modelMPP.x['Hovel PM2'] <= TotalHovelM2*0.6)

#Almonds Composition Constraints
modelMPP.cons.add(modelMPP.x['Chalet AM1'] >= TotalChaletM1*0.4)
modelMPP.cons.add(modelMPP.x['Chalet AM12'] + modelMPP.x['Chalet AM2'] >= TotalChaletM2*0.4)

modelMPP.cons.add(modelMPP.x['Hovel AM1'] >= TotalHovelM1*0.2)
modelMPP.cons.add(modelMPP.x['Hovel AM12'] + modelMPP.x['Hovel AM2'] >= TotalHovelM2*0.2)




In [121]:
SolverFactory('glpk').solve(modelMPP)

{'Problem': [{'Name': 'unknown', 'Lower bound': 40682.5, 'Upper bound': 40682.5, 'Number of objectives': 1, 'Number of constraints': 15, 'Number of variables': 18, 'Number of nonzeros': 72, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.07583951950073242}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [122]:
modelMPP.display()

Model Multiperiod Planning

  Variables:
    x : Size=18, Index={Hovel AM12, Chalet AM2, Chalet PM1, Hovel WM12, Hovel WM2, Chalet WM1, Chalet AM1, Hovel AM1, Hovel PM1, Hovel PM2, Hovel WM1, Hovel PM12, Chalet WM12, Chalet PM2, Hovel AM2, Chalet AM12, Chalet PM12, Chalet WM2}
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
         Chalet AM1 :     0 : 100.0 :  None : False : False : NonNegativeReals
        Chalet AM12 :     0 : 100.0 :  None : False : False : NonNegativeReals
         Chalet AM2 :     0 : 180.0 :  None : False : False : NonNegativeReals
         Chalet PM1 :     0 :  62.5 :  None : False : False : NonNegativeReals
        Chalet PM12 :     0 :   0.0 :  None : False : False : NonNegativeReals
         Chalet PM2 :     0 : 175.0 :  None : False : False : NonNegativeReals
         Chalet WM1 :     0 :  87.5 :  None : False : False : NonNegativeReals
        Chalet WM12 :     0 :   0.0 :  None : False : False : NonNegativeReals
         Chalet WM2 :

### Question 24

Snow Removal

In [51]:
CostSR = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="SRCost",index_col=0)  
CapacitySR = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="SRCapacity",index_col=0)  


In [52]:
CostSR

,A,B,C,D,E
Z-1,3.4,1.4,4.9,7.4,9.3
Z-2,2.4,2.1,8.3,9.1,8.8
Z-3,1.4,2.9,3.7,9.4,8.6
Z-4,2.6,3.6,4.5,8.2,8.9
Z-5,1.5,3.1,2.1,7.9,8.8
Z-6,4.2,4.9,6.5,7.7,6.1
Z-7,4.8,6.2,9.9,6.2,5.7
Z-8,5.4,6.0,5.2,7.6,4.9
Z-9,3.1,4.1,6.6,7.5,7.2
Z-10,3.2,6.5,7.1,6.0,8.3


In [53]:
CapacitySR

,A,B,C,D,E
Z-1,153,153,153,153,153
Z-2,152,152,152,152,152
Z-3,154,154,154,154,154
Z-4,138,138,138,138,138
Z-5,127,127,127,127,127
Z-6,129,129,129,129,129
Z-7,111,111,111,111,111
Z-8,110,110,110,110,110
Z-9,130,130,130,130,130
Z-10,135,135,135,135,135


In [94]:
modelSR = ConcreteModel ("Snow Removal")

Site = CostSR.keys()
Zone = CostSR.index

#DVs

modelSR.x = Var(Zone,Site,domain = Binary)

#Objective Function
modelSR.Cost = Objective(expr = 0.1*1000*sum(sum([modelSR.x[z,s]*CostSR.loc[z,s]*CapacitySR.loc[z,s] for z in Zone]) for s in Site), sense = minimize)

C:\Users\amara\AppData\Local\Temp\ipykernel_35388\2719076199.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelSR.Cost = Objective(expr = 0.1*1000*sum(sum([modelSR.x[z,s]*CostSR.loc[z,s]*CapacitySR.loc[z,s] for z in Zone]) for s in Site), sense = minimize)


In [95]:
modelSR.cons = ConstraintList()

for s in Site:
    modelSR.cons.add(sum([modelSR.x[z,s]*CapacitySR.loc[z,s] for z in Zone]) <= CapacitySR.loc['Max Capacity',s])

for z in Zone:
    modelSR.cons.add(sum([modelSR.x[z,s] for s in Site]) == 1)

In [96]:
SolverFactory('glpk').solve(modelSR)

{'Problem': [{'Name': 'unknown', 'Lower bound': 547000.0, 'Upper bound': 547000.0, 'Number of objectives': 1, 'Number of constraints': 15, 'Number of variables': 50, 'Number of nonzeros': 100, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '73', 'Number of created subproblems': '73'}}, 'Error rc': 0, 'Time': 0.07326459884643555}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [97]:
modelSR.display()

Model Snow Removal

  Variables:
    x : Size=50, Index={Z-2, Z-5, Z-8, Z-9, Z-1, Z-10, Z-4, Z-6, Z-7, Z-3}*{B, A, C, E, D}
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
         ('Z-1', 'A') :     0 :   0.0 :     1 : False : False : Binary
         ('Z-1', 'B') :     0 :   1.0 :     1 : False : False : Binary
         ('Z-1', 'C') :     0 :   0.0 :     1 : False : False : Binary
         ('Z-1', 'D') :     0 :   0.0 :     1 : False : False : Binary
         ('Z-1', 'E') :     0 :   0.0 :     1 : False : False : Binary
        ('Z-10', 'A') :     0 :   0.0 :     1 : False : False : Binary
        ('Z-10', 'B') :     0 :   0.0 :     1 : False : False : Binary
        ('Z-10', 'C') :     0 :   0.0 :     1 : False : False : Binary
        ('Z-10', 'D') :     0 :   1.0 :     1 : False : False : Binary
        ('Z-10', 'E') :     0 :   0.0 :     1 : False : False : Binary
         ('Z-2', 'A') :     0 :   1.0 :     1 : False : False : Binary
         ('Z-2', 'B') : 

### Question 25

Network Flow

In [5]:
CapacityNF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="NFCapacity",index_col=0)  
CostNF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="NFCost",index_col=0)  
DataNF = pd.read_excel('INFO4590P3PyomoData.xlsx',sheet_name="NFData",index_col=0)  


In [6]:
CapacityNF

,Waha,Kiowa,Wharton,Katy,Carthage,Joliet,Henry,Perryville,Lebanon,Maumee,Leidy
Waha,0,25,10,15,0,0,0,0,0,0,0
Kiowa,25,0,0,0,20,0,0,0,0,0,0
Wharton,10,0,0,20,0,0,0,0,0,0,0
Katy,15,0,20,0,30,0,50,0,0,0,0
Carthage,0,20,0,30,0,25,0,30,15,0,0
Joliet,0,0,0,0,25,0,0,15,20,25,0
Henry,0,0,0,50,0,0,0,20,0,0,0
Perryville,0,0,0,0,30,15,20,0,20,0,15
Lebanon,0,0,0,0,15,20,0,20,0,15,30
Maumee,0,0,0,0,0,25,0,0,15,0,25


In [7]:
CostNF

,Waha,Kiowa,Wharton,Katy,Carthage,Joliet,Henry,Perryville,Lebanon,Maumee,Leidy
Waha,0.00,0.35,0.28,0.51,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Kiowa,0.35,0.00,0.00,0.00,0.28,0.00,0.00,0.00,0.00,0.00,0.00
Wharton,0.28,0.00,0.00,0.21,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Katy,0.51,0.00,0.21,0.00,0.42,0.00,0.39,0.00,0.00,0.00,0.00
Carthage,0.00,0.28,0.00,0.42,0.00,0.47,0.00,0.30,0.35,0.00,0.00
Joliet,0.00,0.00,0.00,0.00,0.47,0.00,0.00,0.52,0.45,0.52,0.00
Henry,0.00,0.00,0.00,0.39,0.00,0.00,0.00,0.32,0.00,0.00,0.00
Perryville,0.00,0.00,0.00,0.00,0.30,0.52,0.32,0.00,0.48,0.00,0.22
Lebanon,0.00,0.00,0.00,0.00,0.35,0.45,0.00,0.48,0.00,0.33,0.47
Maumee,0.00,0.00,0.00,0.00,0.00,0.52,0.00,0.00,0.33,0.00,0.53


In [8]:
DataNF

,Waha,Kiowa,Wharton,Katy,Carthage,Joliet,Henry,Perryville,Lebanon,Maumee,Leidy
Supply,0,0,0,100,0,0.00,0,0,0,0,0.00
Demand,0,0,0,0,0,35.00,0,0,0,0,60.00
Price,0,0,0,0,0,4.35,0,0,0,0,4.63


In [10]:
modelNF = ConcreteModel ("Network Flow")

FromP = CapacityNF.index
ToP = CapacityNF.keys()

#DVs

modelNF.x = Var(FromP,ToP,domain = NonNegativeIntegers)


#Objective Function
#Revenue = DataNF.loc['Price',t]*sum(modelNF.x[t,f]-modelNF.x[f,t])] for f in FromP for t in ToP
#Cost = sum(sum([modelNF.x[f,t]*CostNF.loc[f,t] for i in ToP for f in FromP])

modelNF.Profit = Objective(expr = sum(DataNF.loc['Price',t]*(modelNF.x[t,f]-modelNF.x[f,t]) for t in ToP for f in FromP) -
                           sum([sum(modelNF.x[f,t]*CostNF.loc[f,t] for t in ToP) for f in FromP]), sense = maximize)

C:\Users\amara\AppData\Local\Temp\ipykernel_25644\3521469672.py:15: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelNF.Profit = Objective(expr = sum(DataNF.loc['Price',t]*(modelNF.x[t,f]-modelNF.x[f,t]) for t in ToP for f in FromP) -
C:\Users\amara\AppData\Local\Temp\ipykernel_25644\3521469672.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum([sum(modelNF.x[f,t]*CostNF.loc[f,t] for t in ToP) for f in FromP]), sense = maximize)


In [12]:
modelNF.cons = ConstraintList()


for t in ToP:
    modelNF.cons.add(sum(modelNF.x[t,f] - modelNF.x[f,t] for f in FromP) <= DataNF.loc['Demand',t])

for t in ToP:
    modelNF.cons.add(sum(modelNF.x[f,t] - modelNF.x[t,f] for f in FromP) <= DataNF.loc['Supply',t])

for f in FromP:
    for t in ToP:
        modelNF.cons.add(modelNF.x[f,t] <= CapacityNF.loc[f,t])


'pyomo.core.base.constraint.ConstraintList'>) on block Network Flow with a new
Component (type=<class 'pyomo.core.base.constraint.ConstraintList'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


C:\Users\amara\AppData\Local\Temp\ipykernel_25644\2026136847.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelNF.cons.add(sum(modelNF.x[t,f] - modelNF.x[f,t] for f in FromP) <= DataNF.loc['Demand',t])
C:\Users\amara\AppData\Local\Temp\ipykernel_25644\2026136847.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  modelNF.cons.add(sum(modelNF.x[f,t] - modelNF.x[t,f] for f in FromP) <= DataNF.loc['Supply',t])


In [14]:
SolverFactory('glpk').solve(modelNF)

{'Problem': [{'Name': 'unknown', 'Lower bound': 224.6, 'Upper bound': 224.6, 'Number of objectives': 1, 'Number of constraints': 143, 'Number of variables': 121, 'Number of nonzeros': 561, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.049695491790771484}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [15]:
modelNF.display()

Model Network Flow

  Variables:
    x : Size=121, Index={Katy, Lebanon, Leidy, Perryville, Henry, Carthage, Kiowa, Maumee, Wharton, Waha, Joliet}*{Katy, Lebanon, Leidy, Perryville, Henry, Carthage, Kiowa, Maumee, Wharton, Waha, Joliet}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
            ('Carthage', 'Carthage') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
               ('Carthage', 'Henry') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
              ('Carthage', 'Joliet') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                ('Carthage', 'Katy') :     0 :  30.0 :  None : False : False : NonNegativeIntegers
               ('Carthage', 'Kiowa') :     0 :  20.0 :  None : False : False : NonNegativeIntegers
             ('Carthage', 'Lebanon') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
               ('Carthage', 'Leidy') :     0 :   0.0 :  None : False : False : NonN